In [35]:
import pandas as pd
import numpy as np
import seaborn as sb; sb.set()
import matplotlib.pyplot as plt

In [36]:
def make_df(cols, index):
    data = {c: [str(c) + str(i) for i in index] for c in cols}
    return pd.DataFrame(data, index=index)

In [37]:
df1 = make_df('ABC', [1, 2])
df2 = make_df('BCD', [3, 4])
display(df1, df2)

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


# `pd.concat()`
Just like `np.concatenate([[1, 2], [3, 4]], axis = 1)`
1. `axis`: Join to the right or bottom?
2. `ignore_index`: If both df have same indices, do I reindex (ignore) them?
3. `verify_integrity`: If I insist on keeping their indices, give me an error on duplicated indices
4. `join`: I could change to inner
5. `join_axes` (deprecated): I only want these columns/ rows
6. `keys`: do I want to still know which data are from which df? (keys form outer level of multi-index)

In [38]:
np.concatenate([[1, 2], [3, 4]], axis = 0)

array([1, 2, 3, 4])

In [41]:
# Default: inner join; `keys` create MultiIndex
pd.concat([df1, df2], axis = 1, keys = ['x', 'y'])

x              y          
     A    B    C    B    C    D
1   A1   B1   C1  NaN  NaN  NaN
2   A2   B2   C2  NaN  NaN  NaN
3  NaN  NaN  NaN   B3   C3   D3
4  NaN  NaN  NaN   B4   C4   D4

In [22]:
pd.concat([df1, df2], axis = 0, join = 'inner')

,B,C
1,B1,C1
2,B2,C2
1,B3,C3
2,B4,C4


In [28]:
df2.index = [1, 2]
try:
    pd.concat([df1, df2], verify_integrity = True)
except ValueError as e:
    print(f"ValueError: {e}")
    display(pd.concat([df1, df2], ignore_index = False))

ValueError: Indexes have overlapping values: Int64Index([1, 2], dtype='int64')


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
1,NaN,B3,C3,D3
2,NaN,B4,C4,D4


# `pd.append()`: when lazy to write the former
different from lists' append() and extend() 
- Only appends two df at a time, unlike `pd.concat()`
- Creates a new df: hence new index and data buffer. Inefficient  

**Use `pd.concat()` if need to join multiple df**

In [29]:
df1.append(df2)

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
1,NaN,B3,C3,D3
2,NaN,B4,C4,D4


# `pd.merge()`
Built for SQL-like **joins**, unlike `pd.concat()`, which is built to **pile up** rows or columns
1. `on`, `left_on`, `right_on`, `left_index`, `right_index`: On which column?
2. `how`: Default inner join
3. `suffixes`: What if both df have same column names?

In [32]:
df3 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'], 'food': ['fish', 'beans', 'bread']}, columns=['name', 'food'])
df4 = pd.DataFrame({'name': ['Mary', 'Joseph'],'drink': ['wine', 'beer']}, columns=['name', 'drink'])
display(df3, df4)

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


,name,drink
0,Mary,wine
1,Joseph,beer


In [36]:
df4.merge(df3, on='name')

,name,drink,food
0,Mary,wine,bread


In [38]:
df5 = df3.set_index('name')
df4.merge(df5, left_on='name', right_index=True, how='outer')

,name,drink,food
0.0,Mary,wine,bread
1.0,Joseph,beer,NaN
NaN,Peter,NaN,fish
NaN,Paul,NaN,beans


In [41]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [3, 1, 4, 2]})
display(df8, df9)

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [44]:
# suffixes when non-joining columns have same names
pd.merge(df8, df9, on='name', suffixes = ['_L', '_R'])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


# Mock Test

In [49]:
base_link = "https://raw.githubusercontent.com/jakevdp/data-USstates/master/"
pop = pd.read_csv(base_link + 'state-population.csv')
areas = pd.read_csv(base_link + 'state-areas.csv')
abbrevs = pd.read_csv(base_link + 'state-abbrevs.csv')
print(pop.head(), areas.head(), abbrevs.head(), sep = '\n')

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [51]:
# Merge pop and areas based on abbreviation
merged = pd.merge(pop, abbrevs, how = 'outer', left_on = 'state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [52]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [54]:
merged.loc[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [56]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [61]:
# Fill in state values
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [63]:
# Merge with areas df
final = pd.merge(merged, areas, on='state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [64]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [66]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [67]:
final.dropna(inplace = True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [75]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [76]:
data2010.set_index('state', inplace = True)
density = data2010['population']/ data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace = True)
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64